# 回帰と汎化性能
フックの法則（バネの伸びとバネにかかる張力の比例）を例にとって**線形回帰**を学びました。
> **線形回帰** 
> 予測する関数（**回帰関数**と呼びます）が一次関数（**線形関数**ともいいます）であることが予見される時に利用される回帰アルゴリズム

回帰関数が一次関数でないとき、また、回帰関数のタイプが不明である時の回帰問題は、
**カーネル**という手法を用いて解くことができます。

カーネルとはデータ間の類似性を評価する関数（**類似度関数**）であり、
**正定値性**と呼ばれる数学的な性質を満たします。
> データ$x, y$に対して、
カーネルは$x$と$y$の類似性を与える関す$k(x, y)$です。
カーネルは次の用に定義される正定値性を満足します。
>
>任意のデータ$x_1, \dots, x_n$と実数$c_1, \dots, c_n$に対して、次の不等式が成り立つ。
>
> $$
\sum_{i=1}^n\sum_{j=1}^n c_i c_j k(x_i, x_j) \ge 0
$$
> 

SVMによる分類において、万能の類似度関数としてrbfを利用しましたが、
rbfは代表的なカーネルの例です。

## 例1
次のプログラムを実行して、$y = x^2$のグラフを描いてみましょう。

```python
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
def f(x):
    return x**2
x = np.linspace(-3, 3, 1000)
y = f(x)
plt.plot(x, y)
plt.grid()
plt.show()
```

曲線$y = x^2$の周りに誤差を含んで確率的にばらつく50個の訓練データを作成します。
> 各訓練データは$x$座標と$y$座標のペアです。

```python
n = 50
train_x = np.linspace(-1, 1, n)
train_y = f(train_x) + 0.5 * np.random.randn(n)
train_x = train_x.reshape(-1,1) * 3
train_y = train_y.reshape(-1,1) * 3
plt.plot(train_x, train_y, 'o')
plt.grid()
plt.show()
```

次のプログラムは、この訓練データに対して線形回帰を実行します。

プログラムの最終行では**決定係数**を表示します。
決定係数は回帰関数の訓練データへの当てはまり度を表現する指標で、
0から1の間の値を取り、
1に近いほど正確に当てはまっていることを意味します。

```python
reg = LinearRegression()
reg.fit(train_x, train_y)

test_x = np.linspace(-3, 3, 1000).reshape(-1,1)
test_y = reg.predict(test_x)

plt.plot(train_x, train_y, 'o')
plt.plot(test_x, test_y)
plt.grid()
plt.show()
print(reg.score(train_x, train_y))
```

当然ですが、線形回帰ではうまく回帰できないことが分かります。
決定係数も0に近いことが分かる筈です。

一次式ではない回帰関数を探索したいときに役に立つのが**カーネル回帰**です。
カーネル回帰の手法も一つではないのですが、
ここではカーネルリッジ回帰（KernelRidge）を使います。

```python
from sklearn.kernel_ridge import KernelRidge

alpha = 10**-1
gamma = 10**-1

reg = KernelRidge(alpha=alpha, gamma=gamma, kernel='rbf')
reg.fit(train_x, train_y)

test_y = reg.predict(test_x)

plt.plot(train_x, train_y, 'o')
plt.plot(test_x, test_y)
plt.grid()
plt.show()
print(reg.score(train_x, train_y))
```

かなり上手に二次曲線を近似していることが分かると思います。

`alpha`と`gamma`は回帰関数の当てはまり具合を調整する**ハイパーパラメータ**です。

例えば、
```python
alpha = 0
gamma = 10**1
```
として、再度カーネルリッジ回帰を実行して下さい。

前の実行時に比較して、回帰関数の当てはまり度を表す決定係数は1に近くなっており、
訓練データをより正確に表現できるようになっているのですが、
回帰関数はかえって正解の関数（$y = x^2$）から乖離しています。

正解関数からの乖離は予測の性能（**汎化性能**）を悪化させます。

`alpha`と`gamma`の値を変化させて、
汎化性能を最適化すると想われる値を探してみて下さい。

## 例2
もう少し複雑な例を見てみましょう。
次の関数のグラフを$x$の値が$[-3, 3]の区間で描いてみて下さい。
```python
def f(x):
    return x**2 + 5*np.sin(5 * np.pi * x)/np.exp(np.abs(x))
```

次に、この関数から50個の訓練データをサンプリングしますが、$y$の値には誤差が含まれるとします。
> 各訓練データは$x$座標と$y$座標のペアです。

サンプリングには次のプログラムを用います。
```python
n = 50
train_x = np.linspace(-2, 2, n)
train_y = f(train_x) + 0.5 * np.random.randn(n)
```

サンプリングした点をグラフにプロットしてみて下さい。

カーネルリッジ回帰のハイパーパラメータ（`alpha`と`gamma`）の値を変化させて、
正解の関数に近い回帰関数のグラフを描いて下さい。

正解に近い回帰関数を与えるハイパーパラメータの探索は容易ではないことが分かります。

定量的に**ハイパーパラメータ最適化**を実行する方法については、別の機会に学びます。